In [1]:
from __future__ import annotations
import os, sys
import asyncio
from typing import Iterable, Callable, Any
from pathlib import Path

from limes_x.agents import Outpost, Ambassador
from limes_x.agents.data import Clerk, Datastore
from limes_x.utils import KeyGenerator

class LocalStore:
    def __init__(self, outpost: Outpost) -> None:
        self.agent = Datastore(
            self._on_insert,
            self._on_query,
            ({"writeable"}, {}),
            outpost,
        )
        self._keygen = KeyGenerator()
        self._cache = {}
        self._secondary_indexes = {}

    def _new_key(self):
        new_key = None
        while new_key is None or new_key in self._cache:
            new_key = self._keygen.GenerateUID()
        return new_key

    async def _on_insert(self, *args, **kwargs):
        new_key = self._new_key()
        self._cache[new_key] = args[0] if len(args)>0 else None
        for k, v in kwargs.items():
            index = self._secondary_indexes.get(k, {})
            index[v] = index.get(v, []) + [new_key]
            self._secondary_indexes[k] = index

    async def _on_query(self, **query):
        results: set|None = None
        for k, v in query.items():
            hits = self._secondary_indexes.get(k, {}).get(v, [])
            if results is None:
                results = set(hits)
            else:
                results = results.intersection(hits)
        if results is None: results = set()
        return [self._cache[k] for k in results]

outpost = Outpost()
clerk = Clerk(outpost)
temp_store = LocalStore(outpost)

amb = Ambassador(outpost)
r = await amb.RegisterData("val a", dtype="a")
# r = await amb.RegisterComputeModule("./test_modules/")
r

<Ambassador.7fe6fc787ed>:INFORM:[<Clerk.7fe71c779b9>:INFORM:[<Datastore.7fe6fc787e5>:ACKNOWLEDGE:None]]

In [2]:
entry = await temp_store.agent.Query(dtype="a")
entry, temp_store._secondary_indexes, temp_store._cache

(['val a'], {'dtype': {'a': ['j7C3PuB15Zt2']}}, {'j7C3PuB15Zt2': 'val a'})